<a href="https://colab.research.google.com/github/Pointbreak82/Satellite-Image-Time-Series/blob/main/SITSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Pointbreak82/Satellite-Image-Time-Series-.git
!mkdir ~/.kaggle
!cp /content/Satellite-Image-Time-Series-/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d artelabsuper/sentinel2-munich480

Cloning into 'Satellite-Image-Time-Series-'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (10/10), done.
Resolving deltas: 100% (2/2), done.
100% 15.5G/15.5G [02:46<00:00, 106MB/s]
100% 15.5G/15.5G [02:46<00:00, 100MB/s]


In [2]:
!pip install rasterio
!pip install scikit-image
!pip install tqdm
!pip install visdom==0.1.8.4
!pip3 install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.1.8.4-py3-none-any.whl size=226637 sha256=a7fac4ffb4680f5a76c6764beb9f32d460e4b35fe123b9395e968b21751b6030
  Stored in directory: /root/.cache/pip/wheels/68/06/68/a32a0050c901f9c25fd027b9a5bb97e815157090168a1c89aa
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5692 sha256=77d4c22e8235b3127ee0ae22fdc80317a5737f7e650ff3fdd49b6b7d13c5d1c0
  Stored in directory: /root/.cache/pip/wheels/c7/e9/87/1c51daf8e468d5c14931f8ac3344880f903ba96b063675cac2
Successfully built visdom torchfile


In [3]:
import os
for file in os.listdir('/content'):
    print(file)


.config
sentinel2-munich480.zip
Satellite-Image-Time-Series-
sample_data


In [4]:
import os
import zipfile
from tqdm import tqdm

# Specify the name of the downloaded ZIP file
zip_file_name = 'sentinel2-munich480.zip'

# Specify the target directory for extraction
target_directory = '/content/sentinel2-munich480'

# Create the target directory if it doesn't exist
os.makedirs(target_directory, exist_ok=True)

# Get the total number of files in the ZIP archive for progress tracking
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    total_files = len(zip_ref.infolist())

# Extract the contents of the ZIP file with a progress bar
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    for file_info in tqdm(zip_ref.infolist(), desc="Extracting", unit=" files", ncols=100):
        zip_ref.extract(file_info, target_directory)


Extracting: 100%|███████████████████████████████████| 2298627/2298627 [09:10<00:00, 4178.37 files/s]


In [5]:
import os
import torch
import numpy as np
from scipy.ndimage.filters import maximum_filter1d
import random
import rasterio
import torch.nn.functional as F
import torch.utils.data
from torchvision import transforms
LABEL_FILENAME = "y.tif"

def get_dates(path, n=None):
    """
    extracts a list of unique dates from dataset sample

    :param path: to dataset sample folder
    :param n: choose n random samples from all available dates
    :return: list of unique dates in YYYYMMDD format
    """

    files = os.listdir(path)
    dates = list()
    for f in files:
        f = f.split("_")[0]
        if len(f) == 8:  # 20160101
            dates.append(f)

    dates = set(dates)

    if n is not None:
        dates = random.sample(dates, n)

    dates = list(dates)
    dates.sort()
    return dates
data_transforms = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    # Add other transformations as needed
])

class SentinelDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, seqlength=30, tileids=None,input_channels=13,transform=None):
        self.root_dir = root_dir
        self.name = os.path.basename(root_dir)
        self.data_dirs = [d for d in os.listdir(self.root_dir) if d.startswith("data")]
        self.seqlength = seqlength
        self.munich_format = None
        self.src_labels = None
        self.dst_labels = None
        self.unique_labels = np.array([], dtype=float)
        self.b8_index = 3  # Munich dataset
        self.b4_index = 2  # Munich dataset
        self.transform=transform
        stats = dict(
            rejected_nopath=0,
            rejected_length=0,
            total_samples=0
        )

        self.samples = list()
        self.ndates = list()

        dirs = []
        if tileids is None:
            for d in self.data_dirs:
                dirs_name = os.listdir(os.path.join(self.root_dir, d))
                dirs_path = [os.path.join(self.root_dir, d, f) for f in dirs_name]
                dirs.extend(dirs_path)
        else:
            with open(os.path.join(self.root_dir, tileids), 'r') as f:
                files = [el.replace("\n", "") for el in f.readlines()]
            for d in self.data_dirs:
                dirs_path = [os.path.join(self.root_dir, d, f) for f in files]
                dirs.extend(dirs_path)

        self.classids, self.classes = self.read_classes(os.path.join(self.root_dir, "classes.txt"))

        for path in dirs:
            if not os.path.exists(path):
                stats["rejected_nopath"] += 1
                continue
            if not os.path.exists(os.path.join(path, LABEL_FILENAME)):
                stats["rejected_nopath"] += 1
                continue

            ndates = len(self.get_dates(path))

            if ndates < self.seqlength:
                stats["rejected_length"] += 1
                continue

            stats["total_samples"] += 1
            self.samples.append(path)
            self.ndates.append(ndates)

        self.print_stats(stats)

    def read_classes(self, csv):
        with open(csv, 'r') as f:
            classes = f.readlines()

        ids = list()
        names = list()
        for row in classes:
            row = row.replace("\n", "")
            if '|' in row:
                id, cl = row.split('|')
                ids.append(int(id))
                names.append(cl)

        return ids, names

    def get_image_h_w(self):
        label, profile = self.read(os.path.join(self.samples[0], LABEL_FILENAME))
        return label.shape[-2], label.shape[-1]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path = self.samples[idx]
        if path.endswith(os.sep):
            path = path[:-1]
        patch_id = os.path.basename(path)

        label, profile = self.read(os.path.join(path, LABEL_FILENAME))
        profile["name"] = self.samples[idx]

        dates = self.get_dates(path, n=self.seqlength)

        x10 = []
        x20 = []
        x60 = []

        for date in dates:
            if self.munich_format is None:
                self.munich_format = os.path.exists(os.path.join(path, date + "_10m.tif"))
                if self.munich_format:
                    self.b8_index = 3
                    self.b4_index = 2
                else:
                    self.b8_index = 6
                    self.b4_index = 2
            if self.munich_format:
                x10.append(self.read(os.path.join(path, date + "_10m.tif"))[0])
                x20.append(self.read(os.path.join(path, date + "_20m.tif"))[0])
                x60.append(self.read(os.path.join(path, date + "_60m.tif"))[0])
            else:
                x10.append(self.read(os.path.join(path, date + ".tif"))[0])

        x10 = np.array(x10) * 1e-4
        if self.munich_format:
            x20 = np.array(x20) * 1e-4
            x60 = np.array(x60) * 1e-4

        label = label[0]
        self.unique_labels = np.unique(np.concatenate([label.flatten(), self.unique_labels]))
        new = np.zeros(label.shape, np.int)
        for cl, i in zip(self.classids, range(len(self.classids))):
            new[label == cl] = i

        label = new

        label = torch.from_numpy(label)
        x10 = torch.from_numpy(x10)
        if self.munich_format:
            x20 = torch.from_numpy(x20)
            x60 = torch.from_numpy(x60)

            x20 = F.interpolate(x20, size=x10.shape[2:4])
            x60 = F.interpolate(x60, size=x10.shape[2:4])

            x = torch.cat((x10, x20, x60), 1)
        else:
            x = x10

        x = x.permute(1, 0, 2, 3)
        x = x.float()
        label = label.long()

        target_ndvi = self.get_all_signatures(x, label, len(self.classids), self.b4_index, self.b8_index)

        if self.transform:
            x = self.transform(x)
            target_ndvi = self.transform(target_ndvi)

        return x, label, target_ndvi.float(), dates, patch_id

    def read(self, file):
        with rasterio.open(file) as src:
            return src.read(), src.profile

    def get_all_signatures(self, inp, target, num_cls, b4_index, b8_index):
        c, t, h, w = inp.shape
        output_ndvi = np.zeros((t, h, w), dtype=np.float)

        for cls_index_ in range(0, num_cls):
            pts = (target == cls_index_).numpy()
            all_ndvi_x_cls = []
            for row, yr in enumerate(pts):
                for col, xc in enumerate(yr):
                    if xc:
                        b8 = inp[b8_index, :, row, col]
                        b4 = inp[b4_index, :, row, col]
                        ndvi = (b8 - b4) / (b8 + b4)
                        ndvi = np.nan_to_num(ndvi.numpy())
                        all_ndvi_x_cls.append(ndvi)
            mean_ndvi = np.zeros((t,), dtype=float)
            if len(all_ndvi_x_cls) > 1:
                mean_ndvi = np.mean(all_ndvi_x_cls, axis=0)
            if len(all_ndvi_x_cls) == 1:
                mean_ndvi = all_ndvi_x_cls[0]
            mmax_ndvi = self.__max_filter1d_valid(mean_ndvi, 5)
            output_ndvi[:, pts] = mmax_ndvi.reshape(t, 1)

        return torch.from_numpy(output_ndvi).float()

    def __max_filter1d_valid(self, a, w):
        b = a.clip(min=0)
        return maximum_filter1d(b, size=w)

    def get_dates(self, path, n=None):
        files = os.listdir(path)
        dates = list()
        for f in files:
            f = f.split("_")[0]
            if len(f) == 8:
                dates.append(f)

        dates = set(dates)

        if n is not None:
            dates = random.sample(dates, n)

        dates = list(dates)
        dates.sort()
        return dates

    def print_stats(self, stats):
        print_lst = list()
        for k, v in stats.items():
            print_lst.append("{}:{}".format(k, v))
        print('\n', ", ".join(print_lst))

def main():
    root_path = '/content/sentinel2-munich480/munich480'
    sample_duration = 30
    batch_size = 4
    workers = 2  # Reduce the number of workers as suggested by the warning

    traindataset = SentinelDataset(root_path, tileids="tileids/train_fold0.tileids", seqlength=sample_duration)
    traindataloader = torch.utils.data.DataLoader(
        traindataset, batch_size=batch_size, shuffle=True, num_workers=workers)

    # Load test set
    testdataset = SentinelDataset(root_path, tileids="tileids/test_fold0.tileids", seqlength=sample_duration)
    testdataloader = torch.utils.data.DataLoader(
        testdataset, batch_size=batch_size, shuffle=False, num_workers=workers)

    # Load validation set
    validationdataset = SentinelDataset(root_path, tileids="tileids/eval.tileids", seqlength=sample_duration)
    validationdataloader = torch.utils.data.DataLoader(
        validationdataset, batch_size=batch_size, shuffle=False, num_workers=workers)

    # Iterate through data loader to check if the errors are resolved
    for iteration, data in enumerate(traindataloader):
        input, target, target_ndvi, _, _ = data
        print('input temporal series with 30 images of size 13x48x48:', input.shape)
        print('target segmentation image (batchx48x48):', target.shape)
        print('target_ndvi containing 30 channels of size 48x48:', target_ndvi.shape)
        break

if __name__ == "__main__":
    main()


<ipython-input-5-1b28e670a30c>:4: DeprecationWarning: Please use `maximum_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import maximum_filter1d



 rejected_nopath:3059, rejected_length:4778, total_samples:5231

 rejected_nopath:890, rejected_length:1395, total_samples:1747

 rejected_nopath:806, rejected_length:1441, total_samples:1641


<ipython-input-5-1b28e670a30c>:233: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  dates = random.sample(dates, n)
<ipython-input-5-1b28e670a30c>:233: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  dates = random.sample(dates, n)
<ipython-input-5-1b28e670a30c>:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  new = np.zeros(label.shape, np.int)
<ipython-input-5-1b28e670a30c>:158: DeprecationWarning: `np.int` is 

input temporal series with 30 images of size 13x48x48: torch.Size([4, 13, 30, 48, 48])
target segmentation image (batchx48x48): torch.Size([4, 48, 48])
target_ndvi containing 30 channels of size 48x48: torch.Size([4, 30, 48, 48])


In [6]:
import numpy as np
import datetime
import pandas as pd

try:
    from visdom import Visdom
except:
    print("could not find visdom package. try 'pip install visdom'. continue without...")
    Visdom=None
    pass

class Printer():

    def __init__(self, batchsize = None, N = None):
        self.batchsize = batchsize
        self.N = N

        self.last=datetime.datetime.now()

    def print(self, stats, iteration):
        print_lst = list()

        if self.N is None:
            print_lst.append('iteration: {}'.format(iteration))
        else:
            print_lst.append('iteration: {}/{}'.format(iteration, self.N))

        dt = (datetime.datetime.now() - self.last).total_seconds()

        print_lst.append('logs/sec: {:.2f}'.format(dt / 1))

        if self.batchsize is not None:
            print_lst.append('samples/sec: {:.2f}'.format(dt / self.batchsize))

        for k, v in zip(stats.keys(), stats.values()):
            print_lst.append('{}: {:.2f}'.format(k, v))

        print('\r' + ', '.join(print_lst), end="")

        self.last = datetime.datetime.now()


class Logger():

    def __init__(self, columns, modes, csv=None, epoch=0, idx=0):

        self.columns=columns
        self.mode=modes[0]
        self.epoch=epoch
        self.idx = idx
        self.data = pd.DataFrame(columns=["epoch","iteration","mode"]+self.columns)
        self.csv = csv

    def resume(self, data):
        self.data = data
        self.idx = data.index[-1]
        self.epoch = data["epoch"].max()

    def update_epoch(self, epoch=None):
        if epoch is None:
            self.epoch+=1
        else:
            self.epoch=epoch

    def set_mode(self,mode):
        self.mode = mode

    def log(self, stats, iteration):

        stats["epoch"] = self.epoch
        stats["iteration"] = iteration
        stats["mode"] = self.mode

        row = pd.DataFrame(stats, index=[self.idx])

        self.data = self.data.append(row, sort=False)
        self.idx +=1

    def get_data(self):
        return self.data

    def save_csv(self, path=None):
        if path is not None:
            self.data.to_csv(path)
        elif self.csv is not None:
            self.data.to_csv(self.csv)
        else:
            raise ValueError("please provide either path argument or initialize Logger() with csv argument")

class VisdomLogger():
    def __init__(self,**kwargs):
        if Visdom is None:
            self.viz = None # do nothing
            return

        self.viz = Visdom(**kwargs)
        self.windows = dict()

        r = np.random.RandomState(1)
        self.colors = r.randint(0,255, size=(255,3))
        self.colors[0] = np.array([1., 1., 1.])
        self.colors[1] = np.array([0. , 0.18431373, 0.65490196]) # ikb blue

    def update(self, data):
        self.plot_epochs(data)
        self.plot_steps(data)

    def plot_steps(self, data, maxplotpoints=50):

        if not self.viz.check_connection():
            return # do nothing

        name="loss"
        win = "c_"+name
        if "c_"+name in self.windows.keys():
            #win = self.windows["c_"+name]
            update = 'new'
        else:
            #win = None  # first log -> new window
            update = None

        for mode in data["mode"].unique():
            d = data.loc[data["mode"] == mode]

            maxiter = d["iteration"].max()

            if len(d) > maxplotpoints:
                d = d.tail(maxplotpoints)
                #d = d.sample(maxplotpoints).sort_index()

            if maxiter > 0:
                frac_epochs = d["epoch"] + d["iteration"] / maxiter
            else:
                frac_epochs = np.array([0])
            #total_iter = d["epoch"]*maxiter + d["iteration"]
            values = d[name]

            opts = dict(
                title=name,
                showlegend=True,
                xlabel='epochs',
                ylabel=name)

            win = self.viz.line(
                X=frac_epochs,
                Y=values,
                name=mode,
                win=win,
                opts=opts,
                update=update
            )
            update = 'insert'

        self.windows["c_"+name] = win

    def plot_epochs(self, data):
        """
        Plots mean of epochs

        :param data:
        :return:
        """
        if self.viz is None:
            return # do nothing

        if not self.viz.check_connection():
            return # do nothing

        data_mean_per_epoch = data.groupby(["mode", "epoch"]).mean()
        cols = data_mean_per_epoch.columns
        modes = data_mean_per_epoch.index.levels[0]

        for name in cols:

             if name in self.windows.keys():
                 win = self.windows[name]
                 update = 'new'
             else:
                 win = name # first log -> new window
                 update = None

             opts = dict(
                 title=name,
                 showlegend=True,
                 xlabel='epochs',
                 ylabel=name)

             for mode in modes:

                 epochs = data_mean_per_epoch[name].loc[mode].index
                 values = data_mean_per_epoch[name].loc[mode]

                 win = self.viz.line(
                     X=epochs,
                     Y=values,
                     name=mode,
                     win=win,
                     opts=opts,
                     update=update
                 )
                 update='insert'

             self.windows[name] = win


    def plot_images(self, target, output):
        if self.viz is None:
            return

        if not self.viz.check_connection():
            return # do nothing


        # log softmax -> softmax
        output = np.exp(output)

        prediction = np.argmax(output, axis=1)

        target = np.swapaxes(self.colors[target], -1, 1)
        prediction = np.swapaxes(self.colors[prediction], -1, 1)

        self.viz.images(target, win="target", opts=dict(title='Target'))
        self.viz.images(prediction, win="predictions", opts=dict(title='Predictions'))

        b, c, h, w = output.shape
        for cl in range(c):
            arr = np.expand_dims(output[:,cl],1)*255
            self.viz.images(arr, win="class"+str(cl), opts=dict(title="class"+str(cl)))

In [7]:
import sys
import re

class ProgressBar(object):
    DEFAULT = 'Progress: %(bar)s %(percent)3d%%'
    FULL = '%(bar)s %(current)d/%(total)d (%(percent)3d%%) %(remaining)d to go'

    def __init__(self, total, width=40, fmt=DEFAULT, symbol='=',
                 output=sys.stderr):
        assert len(symbol) == 1

        self.total = total
        self.width = width
        self.symbol = symbol
        self.output = output
        self.fmt = re.sub(r'(?P<name>%\(.+?\))d',
            r'\g<name>%dd' % len(str(total)), fmt)

        self.current = 0

    def __call__(self):
        percent = self.current / float(self.total)
        size = int(self.width * percent)
        remaining = self.total - self.current
        bar = '[' + self.symbol * size + ' ' * (self.width - size) + ']'

        args = {
            'total': self.total,
            'bar': bar,
            'current': self.current,
            'percent': percent * 100,
            'remaining': remaining
        }
        print('\r' + self.fmt % args, file=self.output, end='')

    def done(self):
        self.current = self.total
        self()
        print('', file=self.output)

In [8]:
import torch

def save(path, model, optimizer, **kwargs):
    model_state = None
    optimizer_state = None
    if model is not None:
        model_state = model.state_dict()
    if optimizer is not None:
        optimizer_state = optimizer.state_dict()
    torch.save(
        dict(model_state=model_state,
             optimizer_state=optimizer_state,
             **kwargs),
        path
    )


def resume(path, model, optimizer):
    if torch.cuda.is_available():
        snapshot = torch.load(path)
    else:
        snapshot = torch.load(path, map_location="cpu")

    model_state = snapshot.pop('model_state', snapshot)
    optimizer_state = snapshot.pop('optimizer_state', None)

    if model is not None and model_state is not None:
        print("load model")
        model.load_state_dict(model_state)

    if optimizer is not None and optimizer_state is not None:
        optimizer.load_state_dict(optimizer_state)
    return snapshot

In [9]:
import torch.nn as nn
from torch.autograd import Variable
import torch


class ConvLSTMCell(nn.Module):
    def __init__(self, input_size, input_dim, hidden_dim, kernel_size, bias):
        super(ConvLSTMCell, self).__init__()

        self.height, self.width = input_size
        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.padding     = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias        = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        # Ensure that h_cur and c_cur are on the same device as input_tensor
        h_cur = h_cur.to(input_tensor.device)
        c_cur = c_cur.to(input_tensor.device)

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next


    def init_hidden(self, batch_size, input_tensor):
      return (torch.zeros(batch_size, self.hidden_dim, self.height, self.width, device=input_tensor.device),
            torch.zeros(batch_size, self.hidden_dim, self.height, self.width, device=input_tensor.device))




class ConvLSTM(nn.Module):

    def __init__(self, input_size, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim  = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.height, self.width = input_size

        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i-1]

            cell_list.append(ConvLSTMCell(input_size=(self.height, self.width),
                                          input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """

        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful

        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor.permute(1, 0, 2, 3, 4)

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            hidden_state = self._init_hidden(batch_size=input_tensor.size(0))

        layer_output_list = []
        last_state_list   = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):

                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list   = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                    (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [10]:
import torch
import torch.nn
# from models.convlstm.convlstm import ConvLSTMCell
import torch.nn.functional as F

class LSTMSequentialEncoder(torch.nn.Module):
    def __init__(self, height, width, input_dim=30, hidden_dim=64, nclasses=8, kernel_size=(3,3), bias=False):
        super(LSTMSequentialEncoder, self).__init__()

        self.inconv = torch.nn.Conv3d(input_dim, hidden_dim, (1, 3, 3))
        self.conv1 = nn.Conv2d(13, 64, kernel_size=3, stride=1, padding=1)
        self.cell = ConvLSTMCell(input_size=(height, width),
                                 input_dim=hidden_dim,
                                 hidden_dim=hidden_dim,
                                 kernel_size=kernel_size,
                                 bias=bias)

        # Modify the final layer to produce class probabilities
        self.final = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d(1),  # Global average pooling
            torch.nn.Flatten(),
            torch.nn.Linear(hidden_dim, nclasses),
            torch.nn.Softmax(dim=1)  # Apply softmax activation
        )

    def forward(self, x, hidden=None, state=None):

        # (b x t x c x h x w) -> (b x c x t x h x w)
        x = x.permute(0,2,1,3,4)

        x = torch.nn.functional.pad(x, (1, 1, 1, 1), 'constant', 0)
        x = self.inconv.forward(x)

        b, c, t, h, w = x.shape

        if hidden is None:
            hidden = torch.zeros((b, c, h, w))
        if state is None:
            state = torch.zeros((b, c, h, w))

        if torch.cuda.is_available():
            hidden = hidden.cuda()
            state = state.cuda()

        for iter in range(t):

            hidden, state = self.cell.forward(x[:,:,iter,:,:], (hidden, state))

        x = torch.nn.functional.pad(state, (1, 1, 1, 1), 'constant', 0)
        x = self.final.forward(x)

        return F.log_softmax(x, dim=1)


if __name__=="__main__":


    b, t, c, h, w = 2, 10, 3, 320, 320

    model = LSTMSequentialEncoder(height=h, width=w, input_dim=c, hidden_dim=3)

    x = torch.randn((b, t, c, h, w))


    hidden, state = model.forward(x)

In [11]:
import torch

# Create an instance of the model
model = LSTMSequentialEncoder(height=320, width=320, input_dim=30, hidden_dim=64, nclasses=8)

# Generate sample input data
batch_size = 2
seq_length = 10
input_channels = 30
height = 320
width = 320
sample_input = torch.randn((batch_size, seq_length, input_channels, height, width))

# Pass the input data through the model
hidden, state = model.forward(sample_input)

# Examine the output
print("Model output shape:", hidden.shape)


Model output shape: torch.Size([8])


In [12]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from torchvision import transforms

# Define your data transformations
data_transforms = transforms.Compose([

])

data_dir = "/content/sentinel2-munich480/munich480"
# Define your custom dataset
train_dataset = SentinelDataset(data_dir, tileids="tileids/train_fold0.tileids")
test_dataset = SentinelDataset(data_dir, tileids="tileids/test_fold0.tileids")

# Create a custom dataset wrapper to apply transformations
class TransformedDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.dataset[index]
        if self.transform is not None:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.dataset)

# Apply transformations to the datasets
train_dataset = TransformedDataset(train_dataset, transform=data_transforms)
test_dataset = TransformedDataset(test_dataset, transform=data_transforms)
workers=4
# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=workers)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)



 rejected_nopath:3059, rejected_length:4778, total_samples:5231

 rejected_nopath:890, rejected_length:1395, total_samples:1747


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import os
import time
import visdom

# Define your dataset directory (modify this)
data_dir = "/content/sentinel2-munich480/munich480"

# Define hyperparameters and variables
batch_size = 16
num_workers = 0
epochs = 100
learning_rate = 1e-3
snapshot = None
checkpoint_dir = None

# Create a logger
logger = Logger(columns=["loss"], modes=["train", "test"])
vizlogger = VisdomLogger()

# Create your model
nclasses = len(train_dataset.classes)
network = LSTMSequentialEncoder(48, 48, nclasses=nclasses)

# Define your optimizer and loss function
optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)
loss = torch.nn.NLLLoss()

# Move the model and loss to GPU if available
if torch.cuda.is_available():
    network = torch.nn.DataParallel(network).cuda()
    loss = loss.cuda()

# Optionally, load weights from a snapshot
if snapshot is not None:
    state = torch.load(snapshot)

    if "epoch" in state.keys():
        start_epoch = state["epoch"]

    if "data" in state.keys():
        logger.resume(state["data"])

# Define train_epoch and test_epoch functions here
def train_epoch(dataloader, network, optimizer, loss, loggers):
    logger, vizlogger = loggers

    printer = Printer(N=len(dataloader))
    logger.set_mode("train")

    for iteration, data in enumerate(dataloader):
        optimizer.zero_grad()

        inputs, targets = data  # Unpack inputs and targets

        if torch.cuda.is_available():
            inputs = inputs.cuda()
            targets = targets.cuda()

        outputs = network(inputs)
        l = loss(outputs, targets)
        # print(l)
        stats = {"loss": l.item()}  # Use item() to get a scalar value from the tensor

        l.backward()
        optimizer.step()

        printer.print(stats, iteration)
        logger.log(stats, iteration)
        vizlogger.plot_steps(logger.get_data())

def test_epoch(dataloader, network, loss, loggers):
    logger, vizlogger = loggers
    printer = Printer(N=len(dataloader))
    logger.set_mode("test")

    with torch.no_grad():
        for iteration, data in enumerate(dataloader):
            input, target = data

            if torch.cuda.is_available():
                input = input.cuda()
                target = target.cuda()

            output = network.forward(input)
            l = loss(output, target)
            stats = {"loss": l.data.cpu().numpy()}

            printer.print(stats, iteration)
            logger.log(stats, iteration)
            vizlogger.plot_steps(logger.get_data())

        vizlogger.plot_images(target.cpu().detach().numpy(), output.cpu().detach().numpy())

# Start a Visdom server in the background
!python -m visdom.server -port 8097 > /dev/null 2>&1 &

# Wait for a few seconds to ensure the server has started
time.sleep(5)

# Initialize the Visdom client
viz = visdom.Visdom(port=8097)

# Define your custom dataset and data loaders
train_dataset = SentinelDataset(data_dir, tileids="tileids/train_fold0.tileids")
test_dataset = SentinelDataset(data_dir, tileids="tileids/test_fold0.tileids")

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

start_epoch = 0

# Training loop
for epoch in range(start_epoch, epochs):
    logger.update_epoch(epoch)

    print("\nEpoch {}".format(epoch))
    print("train")
    train_epoch(train_dataloader, network, optimizer, loss, loggers=(logger, vizlogger))
    print("\ntest")
    test_epoch(test_dataloader, network, loss, loggers=(logger, vizlogger))

    data = logger.get_data()
    vizlogger.update(data)

    if checkpoint_dir is not None:
        checkpoint_name = os.path.join(checkpoint_dir, "model_{:02d}.pth".format(epoch))
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": network.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": loss,
                "data": data
            },
            checkpoint_name
        )


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 790, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 496, in _make_request
    conn.request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 395, in request
    self.endheaders()
  File "/usr/lib/python3.10/http/client.py", line 1

Exception in user code:
------------------------------------------------------------


ERROR:visdom:[Errno 99] Cannot assign requested address
ERROR:websocket:[Errno 99] Cannot assign requested address - goodbye
ERROR:websocket:error from callback <function Visdom.setup_socket.<locals>.on_close at 0x7bc48a419120>: Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
ERROR:visdom:[Errno 99] Cannot assign requested address
ERROR:websocket:[Errno 99] Cannot assign requested address - goodbye
ERROR:websocket:error from callback <function Visdom.setup_socket.<locals>.on_close at 0x7bc488803250>: Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
ERROR:visdom:[Errno 99] Cannot assign requested address
ERROR:websocket:[Errno 99] Cannot assign requested address - goodbye
ERROR:websocket:error from callback <function Visdom.setup_socket.<locals>.on_close at 0x7bc48a419120>: Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
ERROR:visdom:[Errno 99] Cannot assign requested address
ER

Exception in user code:
------------------------------------------------------------


ERROR:visdom:[Errno 99] Cannot assign requested address
ERROR:websocket:[Errno 99] Cannot assign requested address - goodbye
ERROR:websocket:error from callback <function Visdom.setup_socket.<locals>.on_close at 0x7bc48a419120>: Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
ERROR:visdom:[Errno 99] Cannot assign requested address
ERROR:websocket:[Errno 99] Cannot assign requested address - goodbye
ERROR:websocket:error from callback <function Visdom.setup_socket.<locals>.on_close at 0x7bc488803250>: Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given



 rejected_nopath:3059, rejected_length:4778, total_samples:5231

 rejected_nopath:890, rejected_length:1395, total_samples:1747

Epoch 0
train


<ipython-input-5-1b28e670a30c>:233: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  dates = random.sample(dates, n)
<ipython-input-5-1b28e670a30c>:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  new = np.zeros(label.shape, np.int)
<ipython-input-5-1b28e670a30c>:195: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy

ValueError: ignored